In [25]:
# Save as: msa_site_assoc.py
from Bio import AlignIO
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

# --- NEW: Corrected OR function ---
def corrected_oddsratio(a, b, c, d):
    """Haldane–Anscombe correction for odds ratio when any cell = 0."""
    return ((a + 0.5) * (d + 0.5)) / ((b + 0.5) * (c + 0.5))

# PARAMETERS - adjust as needed
output_file = '/Users/andreaslawaetz/Streptomyces/Angeliga_project/SCOG/amylase_inhibitors/mutated_amylases/group3_tendamistatAmylases_metadata_MSA_STATS.tsv'
msa_fasta = "/Users/andreaslawaetz/Streptomyces/Angeliga_project/SCOG/tree_amylases/alignment/all_secreted_amylases_Group3_w_CMD_seed_SeedsDeleted.fa"
meta_csv = "/Users/andreaslawaetz/Streptomyces/Angeliga_project/SCOG/amylase_inhibitors/mutated_amylases/group3_tendamistat_metadata.csv"

min_non_gap = 10
min_count_per_group = 3
collapse_rare_to_other = True
rare_threshold = 2

# --- read alignment ---
aln = AlignIO.read(msa_fasta, "fasta")
seq_ids = [rec.id for rec in aln]

# --- read metadata ---
meta = pd.read_csv(meta_csv, dtype={'id':str})
meta = meta.set_index('id').loc[seq_ids]
groups = meta['group'].astype(int).values

# --- per-column stats ---
results = []
n_seqs = len(aln)
L = aln.get_alignment_length()

for pos in range(L):
    col = np.array([rec.seq[pos] for rec in aln], dtype=str)
    col = np.where(np.isin(col, list("-.?")), '-', col)

    idx_g0 = np.where(groups == 0)[0]
    idx_g1 = np.where(groups == 1)[0]

    non_gap_mask = col != '-'
    if non_gap_mask.sum() < min_non_gap:
        continue

    counts_all = Counter(col[non_gap_mask])

    if collapse_rare_to_other:
        small = [r for r,c in counts_all.items() if c < rare_threshold and r != '-']
        if small:
            col = np.array([('OTHER' if (x in small) else x) for x in col])
            counts_all = Counter(col[non_gap_mask])

    for residue, total_count in counts_all.items():
        if residue == '-':
            continue

        a = np.sum((col == residue) & (groups == 0))
        b = np.sum((col != residue) & (groups == 0) & (col != '-'))
        c = np.sum((col == residue) & (groups == 1))
        d = np.sum((col != residue) & (groups == 1) & (col != '-'))

        if (a + c) < min_count_per_group:
            continue

        table = np.array([[a, b],
                          [c, d]])

        try:
            oddsratio_raw, pvalue = fisher_exact(table, alternative='two-sided')
        except Exception:
            oddsratio_raw = np.nan
            pvalue = np.nan

        # --- NEW: corrected odds ratio ---
        odds_corr = corrected_oddsratio(a, b, c, d)

        results.append({
            'pos': pos + 1,
            'residue': residue,
            'count_total': int(a + c),
            'count_g0': int(a),
            'count_g1': int(c),
            'n_non_gap': int(non_gap_mask.sum()),
            'a': int(a),
            'b': int(b),
            'c': int(c),
            'd': int(d),
            'oddsratio_raw': float(oddsratio_raw) if np.isfinite(oddsratio_raw) else np.nan,
            'oddsratio_corrected': odds_corr,          # <<< NEW IN OUTPUT
            'pvalue': float(pvalue)
        })

# --- multiple testing + enrichment ---
df = pd.DataFrame(results)

if df.empty:
    print("No sites passed filters. Adjust parameters.")
else:
    df['p_fdr_bh'] = multipletests(df['pvalue'].values, method='fdr_bh')[1]
    df['p_bonf'] = multipletests(df['pvalue'].values, method='bonferroni')[1]

    df['freq_g0'] = df['a'] / (df['a'] + df['b'])
    df['freq_g1'] = df['c'] / (df['c'] + df['d'])
    df['enriched_in'] = np.where(df['freq_g1'] > df['freq_g0'], 'group1', 'group0')

    df = df.sort_values('p_fdr_bh').reset_index(drop=True)
    df.to_csv(output_file, sep='\t', index=False)

    print(f"Wrote {output_file} with {len(df)} tests")


Wrote /Users/andreaslawaetz/Streptomyces/Angeliga_project/SCOG/amylase_inhibitors/mutated_amylases/group3_tendamistatAmylases_metadata_MSA_STATS.tsv with 1862 tests


In [24]:
# Save as: msa_site_assoc.py
from Bio import AlignIO
import pandas as pd
import numpy as np
from collections import Counter
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

# --- NEW: Corrected OR function ---
def corrected_oddsratio(a, b, c, d):
    """Haldane–Anscombe correction for odds ratio when any cell = 0."""
    return ((a + 0.5) * (d + 0.5)) / ((b + 0.5) * (c + 0.5))

# PARAMETERS - adjust as needed
output_file = '/Users/andreaslawaetz/Streptomyces/Angeliga_project/SCOG/amylase_inhibitors/mutated_amylases/group1_3_inhibitor_metadata_MSA_STATS.tsv'
msa_fasta = "/Users/andreaslawaetz/Streptomyces/Angeliga_project/SCOG/tree_amylases/alignment/all_secreted_amylases_Group1_and_3_aligned_w_CMD_seed_seedsDeleted.fa"
meta_csv = "/Users/andreaslawaetz/Streptomyces/Angeliga_project/SCOG/amylase_inhibitors/mutated_amylases/group1_3_inhibitor_metadata.csv"

min_non_gap = 10
min_count_per_group = 3
collapse_rare_to_other = True
rare_threshold = 2

# --- read alignment ---
aln = AlignIO.read(msa_fasta, "fasta")
seq_ids = [rec.id for rec in aln]

# --- read metadata ---
meta = pd.read_csv(meta_csv, dtype={'id':str})
meta = meta.set_index('id').loc[seq_ids]
groups = meta['group'].astype(int).values

# --- per-column stats ---
results = []
n_seqs = len(aln)
L = aln.get_alignment_length()

for pos in range(L):
    col = np.array([rec.seq[pos] for rec in aln], dtype=str)
    col = np.where(np.isin(col, list("-.?")), '-', col)

    non_gap_mask = col != '-'
    if non_gap_mask.sum() < min_non_gap:
        continue

    counts_all = Counter(col[non_gap_mask])

    if collapse_rare_to_other:
        small = [r for r,c in counts_all.items() if c < rare_threshold and r != '-']
        if small:
            col = np.array([('OTHER' if (x in small) else x) for x in col])
            counts_all = Counter(col[non_gap_mask])

    for residue, total_count in counts_all.items():
        if residue == '-':
            continue

        a = np.sum((col == residue) & (groups == 0))
        b = np.sum((col != residue) & (groups == 0) & (col != '-'))
        c = np.sum((col == residue) & (groups == 1))
        d = np.sum((col != residue) & (groups == 1) & (col != '-'))

        if (a + c) < min_count_per_group:
            continue

        table = np.array([[a, b], [c, d]])

        try:
            oddsratio_raw, pvalue = fisher_exact(table, alternative='two-sided')
        except Exception:
            oddsratio_raw = np.nan
            pvalue = np.nan

        odds_corr = corrected_oddsratio(a, b, c, d)

        results.append({
            'pos': pos + 1,
            'residue': residue,
            'count_total': int(a + c),
            'count_g0': int(a),
            'count_g1': int(c),
            'n_non_gap': int(non_gap_mask.sum()),
            'a': int(a),
            'b': int(b),
            'c': int(c),
            'd': int(d),
            'oddsratio_raw': float(oddsratio_raw) if np.isfinite(oddsratio_raw) else np.nan,
            'oddsratio_corrected': odds_corr,
            'pvalue': float(pvalue)
        })

# --- multiple testing + enrichment ---
df = pd.DataFrame(results)

if df.empty:
    print("No sites passed filters. Adjust parameters.")
else:
    df['p_fdr_bh'] = multipletests(df['pvalue'].values, method='fdr_bh')[1]
    df['p_bonf'] = multipletests(df['pvalue'].values, method='bonferroni')[1]

    df['freq_g0'] = df['a'] / (df['a'] + df['b'])
    df['freq_g1'] = df['c'] / (df['c'] + df['d'])
    df['enriched_in'] = np.where(df['freq_g1'] > df['freq_g0'], 'group1', 'group0')

    # --- NEW: extract enriched proteins for each residue-site ---
    enriched_list = []
    for i, row in df.iterrows():
        pos = row['pos'] - 1
        residue = row['residue']
        enriched_group = 1 if row['enriched_in'] == 'group1' else 0

        proteins = [
            seq_ids[j]
            for j in range(n_seqs)
            if groups[j] == enriched_group and aln[j].seq[pos] == residue
        ]

        enriched_list.append(",".join(proteins))

    df['enriched_proteins'] = enriched_list

    # sort for output
    df = df.sort_values('p_fdr_bh').reset_index(drop=True)
    df.to_csv(output_file, sep='\t', index=False)

    print(f"Wrote {output_file} with {len(df)} tests")


Wrote /Users/andreaslawaetz/Streptomyces/Angeliga_project/SCOG/amylase_inhibitors/mutated_amylases/group1_3_inhibitor_metadata_MSA_STATS.tsv with 7285 tests


In [10]:
df[(df['p_fdr_bh'] < 0.05) & (df['enriched_in'] == 'group1')][:100]

,pos,residue,count_total,count_g0,count_g1,n_non_gap,oddsratio,pvalue,p_fdr_bh,p_bonf,freq_g0,freq_g1,enriched_in
0,2554,L,12,5,7,228,0.000000,1.367723e-10,7.478621e-07,7.809699e-07,0.416667,0.583333,group1
1,2341,E,13,6,7,232,0.000000,2.619482e-10,7.478621e-07,1.495724e-06,0.461538,0.538462,group1
3,3455,P,9,3,6,223,0.000000,5.263061e-10,8.156666e-07,3.005208e-06,0.333333,0.666667,group1
5,3083,H,10,4,6,232,0.000000,1.034970e-09,9.849465e-07,5.909679e-06,0.400000,0.600000,group1
9,3194,E,8,3,5,228,0.000000,1.139907e-08,6.531677e-06,6.508872e-05,0.375000,0.625000,group1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,2584,D,3,1,2,233,0.011111,2.297281e-03,4.796332e-02,1.000000e+00,0.333333,0.666667,group1
274,2774,L,3,1,2,232,0.011161,2.317023e-03,4.810983e-02,1.000000e+00,0.333333,0.666667,group1
275,2210,G,3,1,2,231,0.011211,2.337021e-03,4.834924e-02,1.000000e+00,0.333333,0.666667,group1
276,2263,S,3,1,2,230,0.011261,2.357279e-03,4.859229e-02,1.000000e+00,0.333333,0.666667,group1


In [20]:
df[df['count_total'] == df['count_g1']]

,pos,residue,count_total,count_g0,count_g1,n_non_gap,a,b,c,d,oddsratio_raw,oddsratio_corrected,pvalue,p_fdr_bh,p_bonf,freq_g0,freq_g1,enriched_in
45,3238,L,3,0,3,228,0,222,3,3,0.0,0.002247,0.000010,0.001231,0.058580,0.0,0.500000,group1
46,3590,E,3,0,3,220,0,214,3,3,0.0,0.002331,0.000011,0.001231,0.065237,0.0,0.500000,group1
48,3659,T,3,0,3,220,0,214,3,3,0.0,0.002331,0.000011,0.001231,0.065237,0.0,0.500000,group1
50,3204,V,3,0,3,229,0,223,3,3,0.0,0.002237,0.000010,0.001231,0.057812,0.0,0.500000,group1
51,3231,K,3,0,3,230,0,224,3,3,0.0,0.002227,0.000010,0.001231,0.057058,0.0,0.500000,group1
52,3655,E,3,0,3,221,0,215,3,3,0.0,0.002320,0.000011,0.001231,0.064351,0.0,0.500000,group1
54,2965,S,3,0,3,232,0,225,3,4,0.0,0.002851,0.000017,0.001769,0.097281,0.0,0.428571,group1
56,2511,OTHER,3,0,3,228,0,221,3,4,0.0,0.002902,0.000018,0.001799,0.102515,0.0,0.428571,group1
122,3779,T,3,0,3,86,0,80,3,3,0.0,0.006211,0.000195,0.009072,1.000000,0.0,0.500000,group1
135,3305,S,3,0,3,28,0,25,3,0,0.0,0.002801,0.000305,0.012724,1.000000,0.0,1.000000,group1


In [16]:
from scipy.stats import fisher_exact

# === Enter your values here ===
a = 13   # group0 + residue
b = 213   # group0 + other
c = 7    # group1 + residue
d = 0   # group1 + other

# Build the 2×2 table
table = [[a, b],
         [c, d]]

oddsratio, pvalue = fisher_exact(table, alternative='two-sided')

print("Contingency Table:")
print(f"[[{a}, {b}],")
print(f" [{c}, {d}]]")
print()
print("Fisher Exact Test:")
print(f"  Odds ratio = {oddsratio}")
print(f"  P-value    = {pvalue}")


Contingency Table:
[[13, 213],
 [7, 0]]

Fisher Exact Test:
  Odds ratio = 0.0
  P-value    = 1.1477951680283827e-08


In [18]:
import numpy as np
from statsmodels.stats.contingency_tables import Table2x2

table = np.array([[13, 213],
                  [7, 0]])

tbl = Table2x2(table)

print("Corrected OR:", tbl.oddsratio)
print("95% CI:", tbl.oddsratio_confint())


Corrected OR: 0.004359490274983233
95% CI: (np.float64(0.0002343669381724846), np.float64(0.08109145259936938))
